# Regression on state-wise factors and COVID-19 pandemic changes

In [1]:
import pandas as pd
from sklearn.linear_model import LinearRegression, Lasso
from sklearn import preprocessing
from sklearn.linear_model import ElasticNet
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
pd.set_option("display.precision", 4)

## Data preprocessing

In [2]:
df_model = pd.read_csv('model_data_final.csv')

In [3]:
for name in df_model.columns:
  print(name)

Unnamed: 0
state_name
SD
LD
NESC
PC
GS
CPV
start_date
cases_y
infection_rate
deaths_y
death_rate
aging
pop_density_sqkm
education_index
airport_density_sqkm
avg_principle_cost
surgical_quality


In [4]:
df_model.head()

,Unnamed: 0,state_name,SD,LD,NESC,PC,GS,CPV,start_date,cases_y,infection_rate,deaths_y,death_rate,aging,pop_density_sqkm,education_index,airport_density_sqkm,avg_principle_cost,surgical_quality
0,0,California,0,0,0,0,0,0,1/26/20,1.0256,0.1262,0.0000,0.0000,0.143,95.0,13.44,0.0012,1097181,31.2401
1,1,California,1,0,0,0,0,0,3/5/20,17.1667,0.2449,0.3333,0.0035,0.143,95.0,13.44,0.0012,1097181,31.2401
2,2,California,1,0,0,0,1,0,3/11/20,38.0000,0.2639,0.0000,0.0000,0.143,95.0,13.44,0.0012,1097181,31.2401
3,3,California,1,0,0,0,3,0,3/12/20,46.0000,0.2527,1.0000,0.0055,0.143,95.0,13.44,0.0012,1097181,31.2401
4,4,California,1,0,0,1,4,0,3/13/20,63.0000,0.2474,0.5000,0.0018,0.143,95.0,13.44,0.0012,1097181,31.2401


### Drop medicare features for better $R^2$ fitting score

In [5]:
df_XY = df_model.drop(columns=['Unnamed: 0', 'cases_y', 'deaths_y', 'start_date', 'LD', 'avg_principle_cost','surgical_quality'], inplace = False).dropna(inplace=False)
df_XY.head()

,state_name,SD,NESC,PC,GS,CPV,infection_rate,death_rate,aging,pop_density_sqkm,education_index,airport_density_sqkm
0,California,0,0,0,0,0,0.1262,0.0000,0.143,95.0,13.44,0.0012
1,California,1,0,0,0,0,0.2449,0.0035,0.143,95.0,13.44,0.0012
2,California,1,0,0,1,0,0.2639,0.0000,0.143,95.0,13.44,0.0012
3,California,1,0,0,3,0,0.2527,0.0055,0.143,95.0,13.44,0.0012
4,California,1,0,1,4,0,0.2474,0.0018,0.143,95.0,13.44,0.0012


### Check Correlation

In [6]:
df_XY.drop(['state_name'], axis=1, inplace=False).corr(method ='pearson')

,SD,NESC,PC,GS,CPV,infection_rate,death_rate,aging,pop_density_sqkm,education_index,airport_density_sqkm
SD,1.0000,0.3858,0.4228,0.4448,0.2283,-0.2450,-0.1161,0.1413,-0.2529,-0.1211,-0.2349
NESC,0.3858,1.0000,0.4714,0.5040,0.4835,-0.1989,-0.0543,0.0771,-0.0165,0.0297,-0.0201
PC,0.4228,0.4714,1.0000,0.5256,0.4650,-0.2403,0.0245,0.1281,0.1183,0.1493,0.1308
GS,0.4448,0.5040,0.5256,1.0000,0.5144,-0.3442,-0.1505,-0.0888,0.0151,-0.0225,0.0272
CPV,0.2283,0.4835,0.4650,0.5144,1.0000,-0.1183,0.0169,0.2224,-0.2816,-0.2886,-0.2711
infection_rate,-0.2450,-0.1989,-0.2403,-0.3442,-0.1183,1.0000,0.1255,0.1968,0.1395,0.0041,0.1510
death_rate,-0.1161,-0.0543,0.0245,-0.1505,0.0169,0.1255,1.0000,-0.0090,-0.1279,-0.0411,-0.1391
aging,0.1413,0.0771,0.1281,-0.0888,0.2224,0.1968,-0.0090,1.0000,-0.4396,-0.2456,-0.4002
pop_density_sqkm,-0.2529,-0.0165,0.1183,0.0151,-0.2816,0.1395,-0.1279,-0.4396,1.0000,0.8281,0.9879
education_index,-0.1211,0.0297,0.1493,-0.0225,-0.2886,0.0041,-0.0411,-0.2456,0.8281,1.0000,0.8382


### Feature Standardization

In [7]:
var_drop = ['state_name', 'infection_rate', 'death_rate']
df_X_train = df_XY.drop(columns=var_drop)
X = preprocessing.scale(df_X_train.to_numpy())
Y_infection = df_XY['infection_rate'].to_numpy()
Y_death = df_XY['death_rate'].to_numpy()
print('X std {}'.format(X.std(axis=0)))
print('X std {}'.format(X.mean(axis=0)))

X std [1. 1. 1. 1. 1. 1. 1. 1. 1.]
X std [-3.70074342e-17 -3.33066907e-17  3.33066907e-17  7.40148683e-17
  3.70074342e-18 -7.18869408e-16 -1.48029737e-17 -1.11891977e-14
 -2.49800181e-16]


## Model Fitting using Sklearn for $R^2$ score

### Feature VS Infection Rate

In [8]:
# infection
regr = LinearRegression()
infection_res = regr.fit(X, Y_infection)
infection_res.score(X, Y_infection)

0.32962215950599016

### Feature VS Death Rate

In [9]:
# death
regr = LinearRegression()
death_res = regr.fit(X, Y_death)
death_res.score(X, Y_death)

0.12305461109052362

### Save Result

In [10]:
df_coef = pd.DataFrame({'feature': df_X_train.columns, 'coef_infection': infection_res.coef_, 'coef_death': death_res.coef_})
df_coef

,feature,coef_infection,coef_death
0,SD,0.0161,-0.0028
1,NESC,-0.0064,-0.0003
2,PC,-0.0871,0.0035
3,GS,-0.1148,-0.0028
4,CPV,0.0508,0.0006
5,aging,0.1413,-0.0025
6,pop_density_sqkm,0.0555,-0.0009
7,education_index,-0.1977,0.0044
8,airport_density_sqkm,0.2567,-0.0066


In [11]:
df_coef.to_csv('fit_res_sklearn.csv')

## Model fitting using statsmodels for $p-$values

### Append constants

In [12]:
X_sm = sm.add_constant(X)

### Feature VS Infection Rate

In [13]:
model_infection = sm.OLS(Y_infection, X_sm)
# fit linear
infection_res_ols = model_infection.fit()
# fit ElasticNet
infection_res_ols_en = model_infection.fit_regularized()

In [14]:
print(infection_res_ols.summary())

OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.330
Model:                            OLS   Adj. R-squared:                  0.209
Method:                 Least Squares   F-statistic:                     2.732
Date:                Wed, 15 Apr 2020   Prob (F-statistic):             0.0113
Time:                        01:32:21   Log-Likelihood:                -15.081
No. Observations:                  60   AIC:                             50.16
Df Residuals:                      50   BIC:                             71.10
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.4439      0.044     10.089      0.000       0.356       0.532
x

### Feature VS Death Rate

In [15]:
model_death = sm.OLS(Y_death, X_sm)
# fit linear
death_res_ols = model_death.fit()
# fit ElasticNet
death_res_ols_en = model_death.fit_regularized()

In [16]:
print(death_res_ols.summary())

OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.123
Model:                            OLS   Adj. R-squared:                 -0.035
Method:                 Least Squares   F-statistic:                    0.7796
Date:                Wed, 15 Apr 2020   Prob (F-statistic):              0.636
Time:                        01:32:21   Log-Likelihood:                 174.11
No. Observations:                  60   AIC:                            -328.2
Df Residuals:                      50   BIC:                            -307.3
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0060      0.002      3.218      0.002       0.002       0.010
x

### Collect results

In [17]:
df_coef_ols = pd.DataFrame({
    'feature': ['constant'] + list(df_X_train.columns),
    'coef_infection': infection_res_ols.params,
    'p_infection': infection_res_ols.pvalues,
    'coef_infection_en': infection_res_ols_en.params,
    'coef_death': death_res_ols.params,
    'p_death': death_res_ols.pvalues,
    'coef_death_en': death_res_ols_en.params})
df_coef_ols

,feature,coef_infection,p_infection,coef_infection_en,coef_death,p_death,coef_death_en
0,constant,0.4439,1.1905e-13,0.4439,0.0060,0.0023,0.0060
1,SD,0.0161,7.7939e-01,0.0168,-0.0028,0.2568,-0.0028
2,NESC,-0.0064,9.1090e-01,-0.0073,-0.0003,0.9019,-0.0003
3,PC,-0.0871,1.6201e-01,-0.0877,0.0035,0.1846,0.0035
4,GS,-0.1148,7.4530e-02,-0.1141,-0.0028,0.2959,-0.0029
5,CPV,0.0508,4.2399e-01,0.0514,0.0006,0.8198,0.0006
6,aging,0.1413,1.2256e-02,0.1428,-0.0025,0.2814,-0.0026
7,pop_density_sqkm,0.0555,8.5682e-01,0.0856,-0.0009,0.9459,-0.0019
8,education_index,-0.1977,2.4613e-02,-0.1960,0.0044,0.2380,0.0043
9,airport_density_sqkm,0.2567,4.0039e-01,0.2265,-0.0066,0.6102,-0.0056


### **NOTE**: we notice that the coefficients given by fitting with regularization is similar to that without.

In [18]:
df_coef_ols.to_csv("fit_res_sm.csv")